In [4]:
import pandas as pd
from binance.client import Client

api_key = 'your_api_key'
api_secret = 'your_api_secret'
symbol = 'ETHUSDT'
interval = Client.KLINE_INTERVAL_1HOUR  # You can change this to your desired interval

client = Client(api_key, api_secret)

def get_historical_data(symbol, interval, limit=100):
    klines = client.futures_klines(symbol=symbol, interval=interval, limit=limit)
    df = pd.DataFrame(klines, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume', 'close_time', 'quote_asset_volume', 'number_of_trades', 'taker_buy_base_asset_volume', 'taker_buy_quote_asset_volume', 'ignore'])
    df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')
    df.set_index('timestamp', inplace=True)
    df['close'] = pd.to_numeric(df['close'])
    return df

def calculate_rsi(data, window=14, upper_threshold=70, lower_threshold=30):
    # Calculate daily price changes
    delta = data['close'].diff()

    # Calculate gains (positive changes) and losses (negative changes)
    gains = delta.where(delta > 0, 0)
    losses = -delta.where(delta < 0, 0)

    # Calculate average gains and losses over the specified window
    avg_gains = gains.rolling(window=window, min_periods=1).mean()
    avg_losses = losses.rolling(window=window, min_periods=1).mean()

    # Calculate relative strength (RS)
    rs = avg_gains / avg_losses

    # Calculate the RSI
    rsi = 100 - (100 / (1 + rs))

    # Add RSI to the DataFrame
    data['rsi'] = rsi

    # Add signals based on upper and lower thresholds
    data['rsi_signal'] = 'Hold'
    data.loc[data['rsi'] > upper_threshold, 'rsi_signal'] = 'Sell'
    data.loc[data['rsi'] < lower_threshold, 'rsi_signal'] = 'Buy'

    return data

if __name__ == "__main__":
    historical_data = get_historical_data(symbol, interval)
    data_with_rsi = calculate_rsi(historical_data)

    # Print the DataFrame with RSI values and signals
    print(data_with_rsi[['close', 'rsi', 'rsi_signal']])


                       close        rsi rsi_signal
timestamp                                         
2024-01-19 13:00:00  2497.67        NaN       Hold
2024-01-19 14:00:00  2475.47   0.000000        Buy
2024-01-19 15:00:00  2463.66   0.000000        Buy
2024-01-19 16:00:00  2429.11   0.000000        Buy
2024-01-19 17:00:00  2471.78  38.361953       Hold
...                      ...        ...        ...
2024-01-23 12:00:00  2224.80  24.788985        Buy
2024-01-23 13:00:00  2199.51  22.504789        Buy
2024-01-23 14:00:00  2194.98  17.342477        Buy
2024-01-23 15:00:00  2219.07  23.546445        Buy
2024-01-23 16:00:00  2198.65  19.807635        Buy

[100 rows x 3 columns]
